In [14]:
import os.path as osp
import numpy as np
import sys
sys.path.append("/home/ozgucbertug/Documents/GitHub")

from latentHeuristics.src.ae_templates import mlp_architecture_ala_iclr_18, default_train_params
from latentHeuristics.src.autoencoder import Configuration as Conf
from latentHeuristics.src.point_net_ae import PointNetAutoEncoder

from latentHeuristics.src.in_out import snc_category_to_synth_id, create_dir, PointCloudDataSet, \
                                        load_all_point_clouds_under_folder

from latentHeuristics.src.tf_utils import reset_tf_graph
from latentHeuristics.src.general_utils import plot_3d_point_cloud

In [15]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Define Basic Parameters

In [16]:
top_out_dir = '../out/model/'          # Use to save Neural-Net check-points etc.
top_in_dir = '../ghData/data/' # Top-dir of where point-clouds are stored.

experiment_name = 'custom_multi'
n_pc_points = 2048                # Number of points per model.
bneck_size = 128                  # Bottleneck-AE size
ae_loss = 'chamfer'                   # Loss to optimize: 'emd' or 'chamfer'
class_name = "custom_multi"

Load Point Clouds

In [17]:
syn_id = snc_category_to_synth_id()[class_name]
class_dir = osp.join(top_in_dir , syn_id)
all_pc_data = load_all_point_clouds_under_folder(class_dir, n_threads=8, file_ending='.ply', verbose=True)

../ghData/data/00000001/asymmetric_0000.ply
8000 pclouds were loaded. They belong in 1 shape-classes.


Load default training parameters (some of which are listed beloq). For more details please print the configuration object.

    'batch_size': 50   
    
    'denoising': False     (# by default AE is not denoising)

    'learning_rate': 0.0005

    'z_rotate': False      (# randomly rotate models of each batch)
    
    'loss_display_step': 1 (# display loss at end of these many epochs)
    'saver_step': 10       (# over how many epochs to save neural-network)

In [18]:
train_params = default_train_params()

In [19]:
encoder, decoder, enc_args, dec_args = mlp_architecture_ala_iclr_18(n_pc_points, bneck_size)
train_dir = create_dir(osp.join(top_out_dir, experiment_name))

In [20]:
conf = Conf(n_input = [n_pc_points, 3],
            loss = ae_loss,
            training_epochs = train_params['training_epochs'],
            batch_size = train_params['batch_size'],
            denoising = train_params['denoising'],
            learning_rate = train_params['learning_rate'],
            train_dir = train_dir,
            loss_display_step = train_params['loss_display_step'],
            saver_step = train_params['saver_step'],
            z_rotate = train_params['z_rotate'],
            encoder = encoder,
            decoder = decoder,
            encoder_args = enc_args,
            decoder_args = dec_args
           )
conf.experiment_name = experiment_name
conf.held_out_step = 5   # How often to evaluate/print out loss on 
                         # held_out data (if they are provided in ae.train() ).
conf.save(osp.join(train_dir, 'configuration'))

Load Pre-Trained AE Model

In [21]:
load_pre_trained_ae = True
restore_epoch = 500
if load_pre_trained_ae:
    conf = Conf.load(train_dir + '/configuration')
    reset_tf_graph()
    ae = PointNetAutoEncoder(conf.experiment_name, conf)
    ae.restore_model(conf.train_dir, epoch=restore_epoch)

Building Encoder
encoder_conv_layer_0 conv params =  256 bnorm params =  128
Tensor("custom_multi_2/Relu:0", shape=(?, 2048, 64), dtype=float32)
output size: 131072 

encoder_conv_layer_1 conv params =  8320 bnorm params =  256
Tensor("custom_multi_2/Relu_1:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_2 conv params =  16512 bnorm params =  256
Tensor("custom_multi_2/Relu_2:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

encoder_conv_layer_3 conv params =  33024 bnorm params =  512
Tensor("custom_multi_2/Relu_3:0", shape=(?, 2048, 256), dtype=float32)
output size: 524288 

encoder_conv_layer_4 conv params =  32896 bnorm params =  256
Tensor("custom_multi_2/Relu_4:0", shape=(?, 2048, 128), dtype=float32)
output size: 262144 

Tensor("custom_multi_2/Max:0", shape=(?, 128), dtype=float32)
Building Decoder
decoder_fc_0 FC params =  33024 Tensor("custom_multi_2/Relu_5:0", shape=(?, 256), dtype=float32)
output size: 256 

decoder_fc_1 FC par

Build the AE Model

In [22]:
if not load_pre_trained_ae:
    reset_tf_graph()
    ae = PointNetAutoEncoder(conf.experiment_name, conf)

Train the AE Model

In [23]:
if not load_pre_trained_ae:
    buf_size = 1 # Make 'training_stats' file to flush each output line regarding training.
    fout = open(osp.join(conf.train_dir, 'train_stats.txt'), 'a', buf_size)
    train_stats = ae.train(all_pc_data, conf, log_file=fout)
    fout.close()

# Export

Export all latent vectors and labels to csv file

In [31]:
out_dir = "/home/ozgucbertug/Documents/GitHub/latentHeuristics/out"

In [34]:
all_latent_vectors = np.zeros((len(all_pc_data.point_clouds),128))

for i in range(len(all_pc_data.point_clouds)):
    pc = all_pc_data.point_clouds[i].reshape((1,2048,3))
    all_latent_vectors[i] = ae.transform(pc)

if not os.path.exists(out_dir+"/csv"):
        os.makedirs(out_dir+"/csv")

vector_fn = out_dir+"/csv/"+experiment_name+"_all_latent_vectors.csv"
np.savetxt(vector_fn, all_latent_vectors, delimiter=',')

label_fn = out_dir+"/csv/"+experiment_name+"_all_labels.csv"
np.savetxt(label_fn,all_pc_data.labels.reshape((len(all_pc_data.labels,))), delimiter=',', fmt='%s')

Export interpolated point cloud (simple_i --> asymmetric_i, spiral_i, lowPoly_i) to ply file

In [42]:
from plyfile import PlyData, PlyElement

def write_ply(points, filename, text=True):
    points = [(points[i,0], points[i,1], points[i,2]) for i in range(points.shape[0])]
    vertex = np.array(points, dtype=[('x', 'f4'), ('y', 'f4'),('z', 'f4')])
    el = PlyElement.describe(vertex, 'vertex', comments=['vertices'])
    PlyData([el], text=text).write(filename)

if not os.path.exists(out_dir+"/ply"):
        os.makedirs(out_dir+"/ply")

n_samples = 4
n_interpolate = 10

for i in np.linspace(0, 2000, n_samples, endpoint=False):
    i = int(i)
    simple_asymmetric = ae.interpolate(all_pc_data.point_clouds[4000+i],  all_pc_data.point_clouds[0000+i], steps=n_interpolate)
    simple_lowPoly = ae.interpolate(all_pc_data.point_clouds[4000+i],  all_pc_data.point_clouds[2000+i], steps=n_interpolate)
    simple_spiral = ae.interpolate(all_pc_data.point_clouds[4000+i],  all_pc_data.point_clouds[6000+i], steps=n_interpolate)
    for j in range(n_interpolate+2):
        fn = out_dir+"/ply/simple_asymmetric_%4d_%2d" % (i,j)
        write_ply(simple_asymmetric[j], fn)

        fn = out_dir+"/ply/simple_lowPoly_%4d_%2d" % (i,j)
        write_ply(simple_lowPoly[j], fn)

        fn = out_dir+"/ply/simple_spiral_%4d_%2d" % (i,j)
        write_ply(simple_spiral[j], fn)

0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11
0
1
2
3
4
5
6
7
8
9
10
11


Use any plotting mechanism such as matplotlib to visualize the results.